In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, Row, DataFrame
import pydeequ, pyspark
from pyspark.sql.types import *
from pydeequ import Check,CheckLevel
from pydeequ.checks import *
from pydeequ.verification import *
from pydeequ.analyzers import *
from pyspark.sql.functions import lit,current_timestamp
from configparser import ConfigParser
import sys
import calendar,time
from functools import reduce

In [19]:
Parser = ConfigParser()
configFile = r"..\config\config_analysis.json"
Parser.read(configFile)

['..\\config\\config_analysis.json']

In [20]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "C:\Program Files (x86)\PostgreSQL\pgJDBC\postgresql-42.2.18.jar") \
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)\
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)\
    .getOrCreate()

In [21]:
df_merchants = spark.read.format('csv').load(Parser.get('merchants','raw_s3_path'),header=True,inferSchema=True).select('id','state','updated_at','telephone')
df_users = spark.read.format('csv').load(Parser.get('users','raw_s3_path'),header=True,inferSchema=True)

In [22]:
def Result(df, source_table):
    analyzers = Parser.get(source_table,'metrics').split(',')
    analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(eval("Size()"))

    res = lambda fun : analysisResult.addAnalyzer(eval(fun))
    tsp 
    analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, list(map(res,analyzers))[0].run())
    analysisResult_df = analysisResult_df.withColumn('ingestion_tsp',lit(current_timestamp()))
    analysisResult_df = analysisResult_df.withColumn('source_table',lit(source_table))
    analysisResult_df.write.csv(f'..//analyzer_files//{source_table}_{calendar.timegm(time.gmtime())}',header=True,mode='overwrite')

    return analysisResult_df

In [23]:
result = list(map(Result,[df_merchants,df_users],Parser.sections()))
merged_analyzer = reduce(DataFrame.unionAll, result)
merged_analyzer.write.csv(f'..//analyzer_files//merged_analyzer_{calendar.timegm(time.gmtime())}',header=True,mode='overwrite')
merged_analyzer.show()

+-------+-------------+------------+---------+--------------------+------------+
| entity|     instance|        name|    value|       ingestion_tsp|source_table|
+-------+-------------+------------+---------+--------------------+------------+
|Dataset|            *|        Size|   1000.0|2020-12-29 19:18:...|   merchants|
| Column|           id|Distinctness|      1.0|2020-12-29 19:18:...|   merchants|
| Column|           id|Completeness|      1.0|2020-12-29 19:18:...|   merchants|
| Column|           id|     Minimum|    469.0|2020-12-29 19:18:...|   merchants|
| Column|           id|     Maximum| 174577.0|2020-12-29 19:18:...|   merchants|
| Column|    telephone|Completeness|    0.999|2020-12-29 19:18:...|   merchants|
| Column|sign_in_count|     Minimum|      0.0|2020-12-29 19:18:...|       users|
| Column|sign_in_count|     Maximum|4187003.0|2020-12-29 19:18:...|       users|
| Column|        email|Completeness|      1.0|2020-12-29 19:18:...|       users|
|Dataset|            *|     